# Инициализация

Загружаем библиотеки необходимые для выполнения кода ноутбука.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import logging
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

pd.set_option('max_colwidth',130)
pd.set_option('display.max_columns',None)
pd.set_option('display.precision',2)

# === ЭТАП 1 ===

# Загрузка первичных данных

Загружаем первичные данные из файлов:
- tracks.parquet
- catalog_names.parquet
- interactions.parquet

In [2]:
tracks = pd.read_parquet("tracks.parquet")
tracks

,track_id,albums,artists,genres
0,26,"[3, 2490753]",[16],"[11, 21]"
1,38,"[3, 2490753]",[16],"[11, 21]"
2,135,"[12, 214, 2490809]",[84],[11]
3,136,"[12, 214, 2490809]",[84],[11]
4,138,"[12, 214, 322, 72275, 72292, 91199, 213505, 2490809, 6007655, 17294156]",[84],[11]
...,...,...,...,...
999995,101478482,[21399811],[5540395],"[3, 75]"
999996,101490148,[21403052],[9078726],"[11, 20]"
999997,101493057,[21403883],[11865715],"[44, 75]"
999998,101495927,[21404975],[4462686],"[3, 75]"


In [3]:
catalog_names = pd.read_parquet("catalog_names.parquet")
catalog_names

,id,type,name
0,3,album,Taller Children
1,12,album,Wild Young Hearts
2,13,album,Lonesome Crow
3,17,album,Graffiti Soul
4,26,album,Blues Six Pack
...,...,...,...
1812466,101478482,track,На лицо
1812467,101490148,track,Без капли мысли
1812468,101493057,track,SKITTLES
1812469,101495927,track,Москва


In [4]:
interactions = pd.read_parquet("interactions.parquet")
interactions

,user_id,track_id,track_seq,started_at
0,0,99262,1,2022-07-17
1,0,589498,2,2022-07-19
2,0,590262,3,2022-07-21
3,0,590303,4,2022-07-22
4,0,590692,5,2022-07-22
...,...,...,...,...
287,1374582,95514767,288,2022-12-29
288,1374582,97220301,289,2022-12-29
289,1374582,97241171,290,2022-12-29
290,1374582,98542641,291,2022-12-29


# Обзор данных

Проверяем данные, есть ли с ними явные проблемы.

In [5]:
for i in [tracks, catalog_names, interactions]:
    print(i.info(),'\nПропуски:\n',i.isna().sum(),'\n')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 4 columns):
 #   Column    Non-Null Count    Dtype 
---  ------    --------------    ----- 
 0   track_id  1000000 non-null  int64 
 1   albums    1000000 non-null  object
 2   artists   1000000 non-null  object
 3   genres    1000000 non-null  object
dtypes: int64(1), object(3)
memory usage: 30.5+ MB
None 
Пропуски:
 track_id    0
albums      0
artists     0
genres      0
dtype: int64 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1812471 entries, 0 to 1812470
Data columns (total 3 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   id      int64 
 1   type    object
 2   name    object
dtypes: int64(1), object(2)
memory usage: 41.5+ MB
None 
Пропуски:
 id      0
type    0
name    0
dtype: int64 

<class 'pandas.core.frame.DataFrame'>
Index: 222629898 entries, 0 to 291
Data columns (total 4 columns):
 #   Column      Dtype         
---  ------      -----         
 0   user_id    

Пропусков нет, проверим явные дубликаты:

In [6]:
for i in [catalog_names, interactions]:
    print(i.duplicated().sum(),'\n')

0 

0 



In [7]:
tracks['track_id'].duplicated().sum()


0

Дублей так же нет. Таблица tracks содержит списки в ячейках, поэтому в ней проверили токлько track_id. Посмотрим распределения:

In [8]:
for i in [catalog_names, interactions]:
    display(i.describe().style.format(precision=0,thousands=' '))

,id
count,1 812 471
mean,23 216 474
std,25 260 438
min,0
25%,3 480 524
50%,12 114 360
75%,37 738 170
max,101 521 819


,user_id,track_id,track_seq,started_at
count,222 629 898,222 629 898,222 629 898,222 629 898
mean,687 577,36 536 225,462,2022-08-29 16:39:44.541336320
min,0,26,1,2022-01-01 00:00:00
25%,343 371,14 808 489,56,2022-07-02 00:00:00
50%,687 973,35 524 737,181,2022-09-15 00:00:00
75%,1 031 127,56 511 373,506,2022-11-09 00:00:00
max,1 374 582,101 521 819,16 637,2022-12-31 00:00:00
std,396 903,26 617 817,826,nan


Посмотрим распределения track_seq на боксплоте, чтобы понять ситуацию:

In [9]:
# Не тянет
# plt.figure(figsize=(15,6))
# sns.boxplot( interactions['track_seq'], color='#bbccbb' )
# plt.title( 'Плот', fontsize=20, pad=15 );


К сожалению, железо не справляется с 200 млн строк, но это и не проблема, тк у нас нет величин, как-либо связанных с распределениями - есть условные ID и даты, да и всё

# Выводы

Приведём выводы по первому знакомству с данными:
- явных проблем с данными нет,
- корерктирующий действий не понадобилось.

# === ЭТАП 2 ===

# EDA

Наиболее популярные треки, топ-30:

In [10]:
interactions.groupby('track_id',as_index=False).agg({'user_id':'count'}).sort_values(by='user_id',ascending=False)[:30]

,track_id,user_id
9098,53404,111062
483876,33311009,106921
26665,178529,101924
512157,35505245,99490
829320,65851540,86670
368072,24692821,86246
475289,32947997,85886
696106,51241318,85244
90461,795836,85042
647237,45499814,84748


Наиболее популярные жанры, тоже топ-30:

In [11]:
tracks.explode('genres').groupby('genres',as_index=False).agg({'artists':'count'}).sort_values(by='artists',ascending=False)[:30]

,genres,artists
10,11,166285
70,75,128212
95,102,118509
63,68,114245
42,47,72311
2,3,65958
40,44,59778
13,14,55177
15,16,51631
19,20,46707


# Преобразование данных

Преобразуем данные в формат, более пригодный для дальнейшего использования в расчётах рекомендаций.

In [12]:
interactions['started_at'] = pd.to_datetime(interactions['started_at'])  # Преобразование в формат datetime
# Выбор начальной даты (например, начало эпохи UNIX)
start_date = pd.Timestamp('1970-01-01')
# Преобразование дат в количество секунд с начальной даты
interactions['started_at'] = (interactions['started_at'] - start_date).dt.total_seconds() / (24 * 60 * 60)

# Сохранение данных

Сохраним данные в двух файлах в персональном S3-бакете по пути `recsys/data/`:
- `items.parquet` — все данные о музыкальных треках,
- `events.parquet` — все данные о взаимодействиях.

In [ ]:
# Не работает почему-то
# import boto3
# s3 = boto3.client('s3')
# s3.upload_file('tracks.parquet', 's3-student-mle-20250529-05fed48463', 'items.parquet')
# s3.upload_file('interactions.parquet', 's3-student-mle-20250529-05fed48463', 'interactions.parquet')

# Очистка памяти

Здесь, может понадобится очистка памяти для высвобождения ресурсов для выполнения кода ниже. 

Приведите соответствующие код, комментарии, например:
- код для удаление более ненужных переменных,
- комментарий, что следует перезапустить kernel, выполнить такие-то начальные секции и продолжить с этапа 3.

In [14]:
# Пример удаления переменных для освобождения оперативной памяти:
# del interactions, tracks

# === ЭТАП 3 ===

# Загрузка данных

"Если необходимо, то загружаем items.parquet, events.parquet" - нет, этого делать не нужно, ведь они у нас в оперативной памяти

In [15]:
interactions['popularity'] = interactions.groupby('track_id')['user_id'].transform('count')
interactions


,user_id,track_id,track_seq,started_at,popularity
0,0,99262,1,19190.0,2524
1,0,589498,2,19192.0,2392
2,0,590262,3,19194.0,570
3,0,590303,4,19195.0,428
4,0,590692,5,19195.0,246
...,...,...,...,...,...
287,1374582,95514767,288,19355.0,736
288,1374582,97220301,289,19355.0,2950
289,1374582,97241171,290,19355.0,5701
290,1374582,98542641,291,19355.0,3293


# Разбиение данных

Разбиваем данные на тренировочную, тестовую выборки.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(interactions.\
drop('popularity',axis=1), interactions['popularity'], test_size=0.2, random_state=42)

# Топ популярных

Рассчитаем рекомендации как топ популярных.

In [17]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
model = LinearRegression()

# Обучение модели на тренировочной выборке
model.fit(X_train, y_train)

,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [18]:

# Предсказание на тестовой выборке
y_pred = model.predict(X_test)

In [19]:

# Оценка качества модели
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 228863608.09561214


# Персональные

Рассчитаем персональные рекомендации. Для этого объединим данные о треках и их именах:

In [20]:
tracks_with_names = tracks.merge(catalog_names, left_on='track_id', right_on='id')

Подсчитаем количество прослушиваний для каждого трека:

In [21]:

track_popularity = interactions.groupby('track_id').size().reset_index(name='count')

И рекомендуем самые популярные треки:


In [22]:
recommended_tracks = track_popularity.sort_values(by='count', ascending=False)[['track_id']]
print(recommended_tracks[:50])

        track_id
9098       53404
483876  33311009
26665     178529
512157  35505245
829320  65851540
368072  24692821
475289  32947997
696106  51241318
90461     795836
647237  45499814
779281  60292250
197265   6705392
571352  39257277
427048  29544272
6558       37384
699153  51516485
137673   2758009
668078  47627256
39466     328683
757669  57921154
270458  17079396
682662  49961817
731961  54798445
744014  56204557
690843  50685843
504691  34976783
289481  18385776
350335  23559960
630282  43942453
427770  29611476
8553       48951
427412  29569939
709808  52380688
579622  39946957
634046  44184116
940964  83063895
917171  78426489
935913  81848875
23706     148345
330585  21519270
282821  17902234
83145     732401
671909  48591706
79121     694683
23039     137670
906004  76434671
946020  84382282
568244  39089632
790427  61565558
738851  55561798


# Похожие

Рассчитаем похожие, они позже пригодятся для онлайн-рекомендаций. Создаем вектор признаков для каждого трека:

In [ ]:

vectorizer = TfidfVectorizer()
genre_matrix = vectorizer.fit_transform(tracks_with_names['genres'].astype(str))

Рассчитываем косинусное сходство между треками:


In [24]:
similarity_matrix = cosine_similarity(genre_matrix)

Пример вывода похожих треков для первого трека в списке

In [25]:

track_index = 0
similar_tracks = similarity_matrix[track_index].argsort()[::-1]
print("Похожие треки для трека", tracks_with_names['track_id'][track_index])
for i in similar_tracks[1:6]:  # выводим 5 наиболее похожих треков
    print(tracks_with_names['track_id'][i])

Похожие треки для трека 26
26
38
38
26
11134


# Построение признаков

Построим три признака, можно больше, для ранжирующей модели. Первый - это количество прослушиваний для каждого трека:

In [26]:
track_popularity = interactions.groupby('track_id').size().reset_index(name='count')

Подсчитаем частоту взаимодействия с жанрами:

In [ ]:
tracks_with_names['genres'] = tracks_with_names['genres'].apply(lambda x: ', '.join(map(str, x)) if isinstance(x, list) else x)

track_popularity = interactions.groupby('track_id').size().reset_index(name='count')

genre_interactions = interactions.merge(tracks_with_names[['track_id', 'genres']], on='track_id')

TypeError: unhashable type: 'numpy.ndarray'

Третий признак - история прослушивания пользователя:

In [34]:

user_history = interactions[interactions['user_id'] == 'user_id_example']
user_track_history = user_history.merge(tracks_with_names, on='track_id')

# Ранжирование рекомендаций

Построим ранжирующую модель, чтобы сделать рекомендации более точными. Отранжируем рекомендации.

In [59]:
tracks_with_names = tracks.merge(catalog_names, left_on='track_id', right_on='id')
tracks_with_names['quantity'] = tracks_with_names.groupby('track_id')['track_id'].transform('count')

# Создадим признаки для модели
features = ['genres', 'artists', 'albums']
X = tracks_with_names[features]

# Преобразуем категориальные признаки в числовые
X['genres'] = X['genres'].apply(lambda x: ', '.join(list(str(x))))
X['artists'] = X['artists'].apply(lambda x: ', '.join(list(str(x))))
X['albums'] = X['albums'].apply(lambda x: ', '.join(list(str(x))))

# Создаём экземпляр LabelEncoder
le = LabelEncoder()

# Применяем кодирование к каждому из признаков
X['genres'] = le.fit_transform(X['genres'])
X['artists'] = le.fit_transform(X['artists'])
X['albums'] = le.fit_transform(X['albums'])

# Целевая переменная
y = tracks_with_names['quantity']

# Разделим данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Создание и обучение модели
model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
model.fit(X_train, y_train)

# Оценим модель на тестовой выборке
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Точность модели: {accuracy}")

/tmp/ipykernel_2201/2093052237.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['genres'] = X['genres'].apply(lambda x: ', '.join(list(str(x))))
/tmp/ipykernel_2201/2093052237.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['artists'] = X['artists'].apply(lambda x: ', '.join(list(str(x))))
/tmp/ipykernel_2201/2093052237.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the

Точность модели: 0.9536690268593833


# Оценка качества

Проверим оценку качества трёх типов рекомендаций: 

- топ популярных,
- персональных, полученных при помощи ALS,
- итоговых
  
по четырем метрикам: recall, precision, coverage, novelty.

In [61]:
from sklearn.metrics import precision_score, recall_score

# Предположим, что y_test — это истинные метки, а y_pred — предсказания модели
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')

print(f"Точность (Precision): {precision}")
print(f"Полнота (Recall): {recall}")

Точность (Precision): 0.687662647780597
Полнота (Recall): 0.3783081911949608


In [65]:
def calculate_coverage(recommended_items, all_items):
    unique_recommended = set(recommended_items)
    unique_all = set(all_items)
    coverage = len(unique_recommended) / len(unique_all)
    return print(f"Coverage: {coverage}")

calculate_coverage(y_test, tracks_with_names['quantity'])

Coverage: 1.0


In [72]:
def calculate_novelty(recommended_items, item_popularity):
    novelty = 0
    for item in recommended_items:
        if item in item_popularity:
            novelty += 1 / item_popularity[item]
    if recommended_items.any():
        novelty /= len(recommended_items)
    return print(f"Novelty: {novelty}")

calculate_novelty(y_pred,tracks_with_names['quantity'])

Novelty: 0.3333544645360708


# === Выводы, метрики ===

Основные выводы при работе над расчётом рекомендаций, рассчитанные метрики:
- Мы загрузили данные, и проверили их качество - оно оказалось очень хорошим.
- Затем мы немного подготовили данные.
- После этого мы обучили модель и подготовили систему рекомендаций.